<a href="https://colab.research.google.com/github/zunyong01/hello-world/blob/main/lec26_notebook_for_students/swcon425_lec26_notebook_for_students.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/dusdnKR/SWCON425/blob/main/lec26_notebook_for_students/swcon425_lec26_notebook_for_students.ipynb?hl=ko" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 실습 개요

LLM의 학습에는 말을 할 수 있게 만들어주는 Pretraining과 특정한 task를 할 수 있도록 만들어주는 Finetuning이 존재합니다.  
사람으로 치면 아래의 차이와 비슷합니다.

<img src="https://github.com/dusdnKR/SWCON425/blob/main/lec26_notebook_for_students/image/lec26_03.png?raw=1" width="1000px">

앞으로 3차시의 실습을 통해 "말은 하는 상태"의 LLM의 작동 원리를 차근차근 이해하고 구현해보도록 하겠습니다.  
실습은 아래 이미지 기준으로 (1)~(3)까지의 LLM 설계에 대한 내용으로 진행될 예정입니다. (4) Pretraining에 대한 내용은 과제로 나가게 됩니다.

<img src="https://github.com/dusdnKR/SWCON425/blob/main/lec26_notebook_for_students/image/lec26_02.png?raw=1" width="1000px">

# Lec 26 데이터 전처리

실습에 사용될 패키지를 미리 다운로드해주세요. 오픈AI에서 제공하는 BPE 토크나이저 [tiktoken](https://github.com/openai/tiktoken)을 사용할  예정입니다.  

In [ ]:
!pip install tiktoken

In [ ]:
from importlib.metadata import version

print("파이토치 버전:", version("torch")) # 파이토치 버전: 2.8.0+cu126
print("tiktoken 버전:", version("tiktoken")) # tiktoken 버전: 0.11.0

파이토치 버전: 2.0.1+cu118
tiktoken 버전: 0.12.0


우선 데이터 전처리부터 시작하겠습니다.

<img src="https://github.com/dusdnKR/SWCON425/blob/main/lec26_notebook_for_students/image/lec26_01.png?raw=1" width="1000px">

## 2.1 단어 임베딩 이해하기

컴퓨터는 텍스트, 이미지, 오디오 데이터 자체를 인식하지는 못합니다.  
모델의 학습에 이런 여러 모달리티의 데이터를 사용하기 위해서는 컴퓨터가 이해할 수 있는 숫자 형태로 변환해 주어야 합니다.  

데이터를 벡터로 바꾸는 작업을 <b>임베딩</b>이라고 하며, 그 결과를 <b>임베딩 벡터(또는 임베딩)</b>라 부릅니다.  
여러 종류의 임베딩이 있지만 이 실습에서는 텍스트 임베딩만 이용할 예정입니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/02.webp" width="1000px">

## 2.2 텍스트 토큰화하기

LLM 학습에 사용되는 데이터는 매우 많은 양의 텍스트 데이터입니다.  
텍스트 데이터는 데이터 토큰화와 임베딩 과정을 거쳐 모델이 이해할 수 있는 형태로 만들어지게 됩니다.

아래 그림에서와 같이  
(1) <b>문장을 작은 단위(token)로 쪼개고,</b>  
(2) <b>각각을 고유한 Token ID에 매핑한 뒤,</b>  
(3) <b>이를 벡터 표현으로 변환</b>하는 과정을 거치게 됩니다.

<img src="https://github.com/dusdnKR/SWCON425/blob/main/lec26_notebook_for_students/image/lec26_00.png?raw=1" width="800px">

토크나이저가 텍스트 데이터를 쪼개는 방식에는 여러 기준이 있습니다. (character 단위, 단어(word) 단위 등)

<img src="https://github.com/dusdnKR/SWCON425/blob/main/lec26_notebook_for_students/image/lec26_05.png?raw=1" width="800px">

먼저 이해를 돕기 위해 <b>단어와 특수문자</b>를 기준으로만 쪼개보겠습니다.  
이디스 워튼(Edith Wharton)의 단편 소설 "[The Verdict](https://en.wikisource.org/wiki/The_Verdict)"를 텍스트 데이터로 활용하겠습니다.

In [ ]:
import os
import requests

if not os.path.exists("the-verdict.txt"):
    url = (
        "https://raw.githubusercontent.com/rasbt/"
        "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
        "the-verdict.txt"
    )
    file_path = "the-verdict.txt"

    response = requests.get(url, timeout=30)
    response.raise_for_status()
    with open(file_path, "wb") as f:
        f.write(response.content)

with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print(f"총 문자 개수: {len(raw_text)}\n")
print(raw_text[:99])

총 문자 개수: 20479

I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


위 소설을 토큰화하기 전에, 우선 짧은 문장을 이용하여 토크나이저를 구현해 보겠습니다.  
우선 띄어쓰기(`\s`)를 기준으로 나누어 봅시다.

In [ ]:
import re

text = "Hello, world. It's a test."
result = ### your code

print(result)

['Hello,', ' ', 'world.', ' ', "It's", ' ', 'a', ' ', 'test.']


정규식을 이용하면 더 다양한 기준으로 나눌 수 있습니다.  
쉼표(`,`)와 마침표(`.`)를 기준으로도 나눠보겠습니다.

In [ ]:
result = ### your code

print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', "It's", ' ', 'a', ' ', 'test', '.', '']


결과를 확인해보면 띄어쓰기도 포함되어 있는 것을 확인할 수 있습니다.  
추출된 result에서 공백을 삭제하겠습니다.

In [ ]:
result = ### your code
print(result)

['Hello', ',', 'world', '.', "It's", 'a', 'test', '.']


실제 토크나이저는 더 많은 경우를 고려해야 합니다.  
아래 12개의 문장부호도 쪼갤 수 있도록 수정하겠습니다.  

`,`    `.`   `:`   `;`   `?`   `_`   `!`   `"`   `(`   `)`   `'`   `--`

In [ ]:
text = "Hello, world. Is this-- a test?"

result = ### your code
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


예시 문장이 정상적으로 나누어지는 것을 확인할 수 있습니다.  
이제 위에서 만든 토크나이저를 이용해 `raw_text`(단편 소설 'The Verdict')를 토큰화해보겠습니다.

In [ ]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


최종적으로 만들어진 토큰 개수를 확인해보겠습니다.

In [ ]:
print(len(preprocessed))

4690


## 2.3 토큰을 토큰 ID로 변환하기

이렇게 쪼개진 텍스트 토큰은 아직 임베딩 모델이 바로 처리할 수 있는 형태가 아닙니다.  
임베딩 입력으로 들어가 연산에 사용될 수 있도록 어떠한 숫자로 바꿔 주어야 합니다.

이를 위해 각각의 토큰와 대응되는 토큰 ID를 정의하고, 이를 딕셔너리로 저장해 사용하겠습니다.

<img src="https://github.com/dusdnKR/SWCON425/blob/main/lec26_notebook_for_students/image/lec26_04.png?raw=1" width="800px">

저희는 방금 위에서 단편 소설 데이터가 담긴 `raw_text`를 토큰화하고, 그 정보를 `preprocessed`에 할당했습니다.  
`preprocessed`에 대한 중복 제거(`set()`)와 정렬(`sorted()`)을 수행하여 고유한 토큰만 남기도록 하겠습니다.

In [ ]:
all_words = ### your code
vocab_size = len(all_words)

print(vocab_size)

1130


생성된 `all_words`를 토큰 ID를 key, 토큰을 value로 하는 딕셔너리 구조로 만들어 `vacab`에 할당합니다.

In [ ]:
vocab = ### your code

만들어진 `vacab` 어휘사전의 값을 확인해보겠습니다.  
정렬된 토큰 각각에 토큰 ID가 할당되어 있는 것을 확인할 수 있습니다.

In [ ]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 20:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)


방금 실습한 내용을 모두 합쳐서 토큰화 클래스를 만들어 보도록 하겠습니다.

- `SimpleTokenizerV1`는 사전 정의된 어휘사전을 기준으로 작동합니다.  
- `encode(text)`는 입력된 텍스트를 토큰화하여 토큰 ID로 바꿔주고,  
- `decode(ids)`는 반대로 입력된 토큰 ID를 텍스트로 바꾸어 줍니다.

In [ ]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = ### your code
        self.int_to_str = ### your code

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)

        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = ### your code
        return ids

    def decode(self, ids):
        text = ### your code
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text) # 각 토큰에 모두 띄어쓰기가 되어 있기 때문에, 구둣점 문자 앞의 공백을 따로 제거해주어야 합니다.
        return text

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/08.webp?123" width="1000px">

만들어진 토크나이저를 이용하여 문자열을 토큰 ID로 바꾸어 보겠습니다.

In [ ]:
tokenizer = ### your code

text = """"It's the last he painted, you know,"
           Mrs. Gisburn said with pardonable pride."""
ids = ### your code
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


디코더를 이용하여 토큰 ID를 다시 문자열로 디코딩할 수 있습니다.

In [ ]:
decoded_text = ### your code
print(decoded_text)

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


다른 문장도 인코딩/디코딩 해보도록 하겠습니다.

In [ ]:
text = "Hello!"

tokenizer.decode(tokenizer.encode(text))

KeyError: 'Hello'

오류가 나는 이유는 <b>어휘사전에 정의되지 않은 단어</b>를 토큰화하려고 했기 때문입니다.


이런 경우를 처리하기 위해 토크나이저에는 <b>특수 토큰</b>이 존재합니다.  
특수 토큰은 모르는 단어 처리 외에도 텍스트의 시작과 끝을 나타낼 때 사용되는 등 특수한 상황에 사용됩니다.

## 2.4 특수 문맥 토큰 추가하기

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/09.webp?123" width="1000px">

일부 토크나이저는 특수 토큰을 사용해 LLM이 추가적인 맥락을 이해하도록 돕습니다.  
보편적으로 사용되는 특수 토큰에는 다음과 같은 것들이 있습니다.
- `[BOS]` (beginning of sequence)는 텍스트의 시작을 표시합니다
- `[EOS]` (end of sequence)는 텍스트의 끝을 표시합니다 (관련이 없는 여러 텍스트를 연결할 때 사용)
- `[PAD]` (padding) 하나 이상의 배치 크기로 LLM을 훈련할 때 배치 안에 길이가 다른 텍스트가 포함될 수 있습니다. 모든 텍스트의 길이를 동일하게 맞추기 위해 짧은 텍스트에 패딩 토큰을 더하여 가장 긴 텍스트의 길이에 맞춰줄 때 사용합니다.
- `[UNK]`는 어휘 사전에 없는 단어를 나타냅니다.

GPT-2 모델은 복잡도를 줄이기 위해 `<|endoftext|>` 토큰만 사용합니다.
- `<|endoftext|>`는 기본적으로 위에서 언급한 `[EOS]` 토큰과 같은 역할을 합니다.
- GPT는 `<|endoftext|>`를 패딩에도 사용합니다. (배치 입력에서 모델을 훈련할 때 마스크를 사용하기 때문에 어차피 가려짐, 마스킹에 대해서는 추후 설명 예정)
- GPT-2는 어휘사전에 없는 토큰을 위해 `<UNK>` 토큰을 사용하지 않습니다. GPT-2는 바이트 페어 인코딩(BPE) 토크나이저를 사용해 단어를 더 작은 부분단어로 분할합니다. (추후 설명 예정)

아래는 `<|endoftext|>` 토큰의 사용 예시입니다.  
두 개의 독립된 텍스트 사이에 `<|endoftext|>` 토큰을 사용하여 구분합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/10.webp" width="1000px">

저희의 어휘사전에도 특수 토큰을 추가해보도록 하겠습니다.  
알지 못하는 단어를 표현하는 `<|unk|>`와 GPT-2 훈련에서 텍스트의 끝을 나타내기 위해 사용된 `<|endoftext|>` 토큰을 사용하겠습니다.

In [ ]:
all_tokens = sorted(list(set(preprocessed)))
### your code

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [ ]:
len(vocab.items())

1132

특수 토큰이 잘 추가되었는지 확인해보겠습니다.  
어휘사전은 오름차순으로 정렬된 상태이기 때문에 `<`로 시작하는 특수 토큰들은 맨 뒤에 추가되었을 것입니다.

In [ ]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


인코딩 과정에서 새로운 단어가 나올 경우 `<unk>` 토큰이 할당되도록 토크나이저를 수정하겠습니다.

In [ ]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = ### your code

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

수정된 토크나이저로 텍스트를 토큰화해 보겠습니다.  
추가로, 두 문장 사이에 `<|endoftext|>` 특수 토큰도 넣도록 하겠습니다.

In [ ]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello!"
text2 = "Dobby wants to go home."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello! <|endoftext|> Dobby wants to go home.


위 문장을 토큰 ID로 인코딩해보겠습니다.  
출력 결과에서 새롭게 정의한 특수 토큰의 ID를 찾아볼 수 있습니다.

- `<|endoftext|>` ↔ 1130
- `<|unk|>` ↔ 1131

In [ ]:
tokenizer.encode(text)

[1131, 0, 1130, 1131, 1076, 1016, 497, 552, 7]

인코딩 결과를 다시 디코딩해서 확인해보면, 특수 토큰의 사용 방식을 쉽게 이해할 수 있습니다.

In [ ]:
decoded_text = tokenizer.decode(tokenizer.encode(text))

print(f"Input text:\n{text}\n")
print(f"Decoded text:\n{decoded_text}")

Input text:
Hello! <|endoftext|> Dobby wants to go home.

Decoded text:
<|unk|>! <|endoftext|> <|unk|> wants to go home.


## 2.5 바이트 페어 인코딩

GPT-2는 바이트 페어 인코딩([BPE](https://github.com/openai/gpt-2/blob/master/src/encoder.py)) 토크나이저를 사용하기 때문에 `<|unk|>` 토큰을 사용하지 않습니다.  
BPE는 단어를 더 작은 문자로 분할하여 처리하기 때문에 모르는 단어가 나오더라도 여러 토큰을 합쳐서 표현할 수 있습니다.

(예시) "unfamiliarword" → ["unfam", "iliar", "word"] (BPE의 훈련에 따라 결과가 달라질 수 있음)

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/11.webp" width="1000px">

본 실습에서는서는 오픈AI의 오픈 소스 [tiktoken](https://github.com/openai/tiktoken) 라이브러리에서 제공하는 BPE 토크나이저를 사용하겠습니다.  
이 라이브러리는 계산 성능을 높이기 위해 핵심 알고리즘을 러스트(Rust)로 구현합니다.

In [ ]:
import tiktoken

tokenizer = ### your code

불러온 토크나이저의 특수 토큰 종류를 확인해보겠습니다.  
GPT-2는 특수 토큰으로 `<|endoftext|>`만을 사용합니다.

In [ ]:
tokenizer.special_tokens_set

{'<|endoftext|>'}

GPT-2의 토크나이저를 이용해 인코딩/디코딩을 수행해보겠습니다.

In [ ]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace."
)

integers = ### your code
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 27271, 13]


`allowed_special` 파라미터는 지정된 특수 토큰만 사용할 수 있도록 해줍니다.  
GPT-2는 하나의 특수 토큰만 가지고 있기 때문에 고려하지 않아도 괜찮습니다.  
모두 사용했을 때와 `<|endoftext|>`만 지정했을 때의 인코딩 결과는 동일합니다.

In [ ]:
print(tokenizer.encode(text, allowed_special={"<|endoftext|>"}))
print(tokenizer.encode(text, allowed_special='all'))

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 27271, 13]
[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 27271, 13]


각 단어가 어떻게 토큰화되었는지 알아보겠습니다.

In [ ]:
decoded_token = []

for token_id in integers:
    decoded_token.append(tokenizer.decode([token_id]))

print(text)
print(decoded_token)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace.
['Hello', ',', ' do', ' you', ' like', ' tea', '?', ' ', '<|endoftext|>', ' In', ' the', ' sun', 'lit', ' terr', 'aces', ' of', ' some', 'unknown', 'Place', '.']


단어 하나가 토큰 하나로 표현되는 경우도 있지만, 아래 단어의 경우 더 여러 토큰의 집합으로 표현되었습니다.

- `sunlit` → `sun` / `lit`
- `terraces` → `terr` / `aces`
- `someunknownPlace` → `some` / `unknown` / `Place`

## 2.6 슬라이딩 윈도로 데이터 샘플링하기

초기의 LLM은 주어진 토큰을 기반으로 다음 토큰을 예측하도록 학습시킬 수 있습니다.  
이를 <b>next token prediction</b>이라고 합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/12.webp" width="1000px">

LLM이 next token prediction task를 수행하도록 하기 위해서는 훈련 데이터가 필요합니다.  
아까의 단편 소설 데이터를 "주어진 토큰"과 "다음 토큰"으로 나누어 input(x)과 target(y) 데이터셋을 준비하겠습니다.

In [ ]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = ### your code
print(len(enc_text))

5145


슬라이싱 방식을 쉽게 이해하기 위해 소설의 중간 부분을 슬라이싱하여 테스트에 사용하겠습니다.

In [ ]:
enc_sample = enc_text[21:]

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/13.webp?123" width="1000px">

input(파란색) 데이터를 기반으로 target(빨간색) 데이터를 출력하도록 학습해야 합니다.  
간단하게 확인해보기 위해 위의 이미지와 같이 `context_size`를 작게 잡고 `x`와 `y`를 출력해보겠습니다.

In [ ]:
context_size = 4

x = ### your code
y = ### your code

print(f"x: {x}")
print(f"y:      {y}")

x: [340, 373, 645, 1049]
y:      [373, 645, 1049, 5975]


데이터셋을 이용해 next token prediction 과정을 출력하여 확인해보겠습니다.  
좌측의 데이터가 input, 우측의 데이터가 target 토큰을 의미합니다.

In [ ]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

[340] ----> 373
[340, 373] ----> 645
[340, 373, 645] ----> 1049
[340, 373, 645, 1049] ----> 5975


토큰 ID(`int`)가 아닌 토큰(`str`)으로도 확인해보겠습니다.

In [ ]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 it ---->  was
 it was ---->  no
 it was no ---->  great
 it was no great ---->  surprise


추후 학습을 위해 입력 데이터셋을 돌면서 input과 target을 반환하는 간단한 데이터 로더를 구현하겠습니다.

`GPTDatasetV1`은 입력된 텍스트 데이터를 토큰화하고, 슬라이딩 윈도를 이용해 데이터를 `max_length` 길이로 나눕니다.  
`stride`를 이용해 데이터 간 단어가 겹치는 정도를 조절할 수 있습니다.

아래 이미지는 `max_length`가 4일 때, `stride`를 1과 4로 설정했을 때의 차이를 보여줍니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/14.webp" width="1000px">

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # 전체 텍스트를 토큰화합니다.
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "토큰화된 입력의 개수는 적어도 max_length+1과 같아야 합니다."

        # 슬라이딩 윈도를 사용해 책을 max_length 길이의 중첩된 시퀀스로 나눕니다.
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = ### your code
            target_chunk = ### your code
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [ ]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    tokenizer = ### your code

    dataset = ### your code

    dataloader = ### your code

    return dataloader

`max_length`가 4이고, `batch_size`가 1인 설정한 데이터 로더를 호출하여 결과를 확인해보겠습니다.  
`batch_size`는 모델을 학습시킬 때 사용되는 데이터의 묶음 단위로, 현재는 1로 설정했기 때문에 `x`, `y`가 각각 1개씩만 반환됩니다.

In [ ]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [ ]:
dataloader = ### your code

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\user\miniconda3\envs\llmfromscratch\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\user\miniconda3\envs\llmfromscratch\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\user\miniconda3\envs\llmfromscratch\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\user\miniconda3\envs\llmfromscratch\lib\site-packages\traitlets\config\application.py", line 1075, 

In [ ]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


실제 학습 시에는 여러 데이터를 한 번에 처리하게 됩니다.  
`batch_size`를 늘려서 여러 데이터쌍을 한 번에 가져오도록 하겠습니다.  
배치 간에 겹치는 데이터가 많으면 모델이 학습 데이터에 과도하게 특화되는 <b>overfitting</b>이 발생할 수 있으므로 `stride`를 4로 증가시키겠습니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/14.webp" width="1000px">

In [ ]:
dataloader = ### your code

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("input:\n", inputs)
print("\ntarget:\n", targets)

input:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

target:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## 2.7 토큰 임베딩 만들기

이제 임베딩 과정을 거쳐서 토큰 ID를 벡터 표현으로 변환해주어야 합니다.

일반적으로 임베딩 층은 LLM 모델의 일부이며, 모델 훈련 과정에서 업데이트(훈련)됩니다.  
잘 학습된 임베딩 층은 벡터 공간 내에서 <b>비슷한 단어는 가깝게, 비슷하지 않은 단어는 멀게</b> 위치하도록 매핑할 수 있어야 합니다.

<img src="https://github.com/dusdnKR/SWCON425/blob/main/lec26_notebook_for_students/image/lec26_06.png?raw=1" width="1000px">

간단한 예시로 6개 단어로 구성된 어휘사전에 대한 3차원 임베딩을 가지는 임베딩 층(Embedding layer)을 만들겠습니다.  
`torch`를 이용해 구현하도록 하겠습니다.

In [ ]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = ### your code

임베딩 층의 가중치를 확인해보면, 6x3 가중치 행렬이 만들어진 것을 확인할 수 있습니다.  
n번 행은 어휘사전에 있는 n번째 단어의 임베딩 벡터를 의미합니다.

In [ ]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


이제, 토큰 ID가 2, 3, 5, 1인 샘플이 있다고 가정하고 방금 구현한 임베딩 층으로 임베딩해보도록 하겠습니다.

In [ ]:
input_ids = torch.tensor([2, 3, 5, 1])

print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


출력 결과는 `embedding_layer` 가중치 행렬의 2, 3, 5, 1번째 행과 정확히 같습니다. (0부터 시작)  
이와 같이 임베딩 층은 기본적으로 순서에 맞는 행을 별도의 계산 없이 그대로 가져오는 <b>룩업 연산</b>을 수행하도록 설계되어 있습니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/16.webp?123" width="1000px">

## 2.8 단어 위치 인코딩하기

임베딩 층은 토큰 ID를 문장 내에서의 위치와 관계 없이 동일한 벡터 표현으로 바꿉니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/17.webp" width="1000px">

하지만, 우리가 사용하는 자연어는 어순에 큰 영향을 받습니다.  
어순을 고려하기 위해 단어의 위치에 따른 <b>위치 임베딩(positional embedding)</b>을 정의하여 토큰 임베딩 결과와 함께 사용합니다.

아래 예시에서처럼 토큰 임베딩과 위치 임베딩을 더해 최종 입력 임베딩으로 사용합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/18.webp" width="1000px">

실제 트랜스포머는 위의 그림처럼 1.1, 1.2, ...처럼 정수 등차수열을 더하지는 않습니다.  
문장의 길이가 길어질 경우 더해지는 값이 너무 커져서 토큰 임베딩 값이 가려질 수 있기 때문입니다.

<img src="https://github.com/dusdnKR/SWCON425/blob/main/lec26_notebook_for_students/image/lec26_07.png?raw=1" width="1000px">

대신 이처럼 삼각함수를 이용하여 정의한 계산식을 사용하게 됩니다. (참고로만 봐주세요)

이제 단편 소설 데이터를 인코딩해보도록 하겠습니다.

우선, 임베딩 층을 정의해주어야 합니다.  
BPE 인코더의 어휘사전 크기인 50,257을 행의 크기로 하고, 입력 토큰은 256차원의 벡터 표현으로 인코딩한다고 가정하겠습니다.

In [ ]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = ### your code

데이터 로더에서 데이터를 샘플링한 다음 배치에 있는 각 샘플의 토큰을 256차원 벡터로 임베딩하겠습니다.  
`batch_size`가 8이고 샘플마다 4개의 토큰이 있다면(아까 `max_length` 4로 설정함) 8 x 4 x 256 크기의 텐서가 만들어 집니다.

In [ ]:
max_length = 4
dataloader = ### your code
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [ ]:
print("inputs:\n", inputs)
print("\n입력 크기:\n", inputs.shape)

inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

입력 크기:
 torch.Size([8, 4])


In [ ]:
token_embeddings = ### your code
print(token_embeddings.shape)
print(token_embeddings)

torch.Size([8, 4, 256])
tensor([[[ 0.4913,  1.1239,  1.4588,  ..., -0.3995, -1.8735, -0.1445],
         [ 0.4481,  0.2536, -0.2655,  ...,  0.4997, -1.1991, -1.1844],
         [-0.2507, -0.0546,  0.6687,  ...,  0.9618,  2.3737, -0.0528],
         [ 0.9457,  0.8657,  1.6191,  ..., -0.4544, -0.7460,  0.3483]],

        [[ 1.5460,  1.7368, -0.7848,  ..., -0.1004,  0.8584, -0.3421],
         [-1.8622, -0.1914, -0.3812,  ...,  1.1220, -0.3496,  0.6091],
         [ 1.9847, -0.6483, -0.1415,  ..., -0.3841, -0.9355,  1.4478],
         [ 0.9647,  1.2974, -1.6207,  ...,  1.1463,  1.5797,  0.3969]],

        [[-0.7713,  0.6572,  0.1663,  ..., -0.8044,  0.0542,  0.7426],
         [ 0.8046,  0.5047,  1.2922,  ...,  1.4648,  0.4097,  0.3205],
         [ 0.0795, -1.7636,  0.5750,  ...,  2.1823,  1.8231, -0.3635],
         [ 0.4267, -0.0647,  0.5686,  ..., -0.5209,  1.3065,  0.8473]],

        ...,

        [[-1.6156,  0.9610, -2.6437,  ..., -0.9645,  1.0888,  1.6383],
         [-0.3985, -0.9235, -1.31

만들어진 토큰 임베딩에 위치 임베딩을 더해보겠습니다.

위치 임베딩을 정의하는 방법은 여러 가지가 있으며, GPT-2는 절대 위치 임베딩을 사용합니다.  
본 실습에서는 간단하게 입력 텍스트의 최대 길이(`max_length`)를 행으로 하는 임베딩 층을 정의하여 위치 임베딩으로 사용하겠습니다.

위의 토큰 임베딩 결과에 위치 임베딩을 더해야 하므로 벡터 차원은 동일하게 256(=`output_dim`)으로 설정합니다.

In [ ]:
torch.manual_seed(456)
context_length = ### your code
pos_embedding_layer = ### your code

print(pos_embedding_layer.weight)

Parameter containing:
tensor([[ 0.8035, -0.0746, -0.9457,  ..., -0.2986,  0.2751,  0.7839],
        [ 0.3819, -1.0562,  0.3459,  ..., -0.4154,  0.3381,  0.5439],
        [-0.4568, -0.6499,  0.4741,  ...,  1.0376,  0.1310,  0.9778],
        [ 1.1472,  2.0686, -0.1244,  ...,  1.0887,  0.1438,  0.7459]],
       requires_grad=True)


In [ ]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))

print(pos_embeddings.shape)
print(pos_embeddings)

torch.Size([4, 256])
tensor([[ 0.8035, -0.0746, -0.9457,  ..., -0.2986,  0.2751,  0.7839],
        [ 0.3819, -1.0562,  0.3459,  ..., -0.4154,  0.3381,  0.5439],
        [-0.4568, -0.6499,  0.4741,  ...,  1.0376,  0.1310,  0.9778],
        [ 1.1472,  2.0686, -0.1244,  ...,  1.0887,  0.1438,  0.7459]],
       grad_fn=<EmbeddingBackward0>)


LLM에 사용될 입력 임베딩을 만들기 위해 토큰 임베딩과 위치 임베딩을 더합니다.

In [ ]:
input_embeddings = ### your code

print(input_embeddings.shape)
print(input_embeddings)

torch.Size([8, 4, 256])
tensor([[[ 1.2948,  1.0493,  0.5132,  ..., -0.6981, -1.5985,  0.6394],
         [ 0.8300, -0.8026,  0.0804,  ...,  0.0843, -0.8610, -0.6406],
         [-0.7075, -0.7045,  1.1428,  ...,  1.9993,  2.5046,  0.9250],
         [ 2.0929,  2.9344,  1.4947,  ...,  0.6342, -0.6022,  1.0943]],

        [[ 2.3495,  1.6623, -1.7304,  ..., -0.3990,  1.1335,  0.4418],
         [-1.4802, -1.2476, -0.0353,  ...,  0.7066, -0.0114,  1.1529],
         [ 1.5279, -1.2981,  0.3326,  ...,  0.6535, -0.8046,  2.4257],
         [ 2.1119,  3.3660, -1.7451,  ...,  2.2349,  1.7235,  1.1428]],

        [[ 0.0322,  0.5826, -0.7794,  ..., -1.1030,  0.3292,  1.5265],
         [ 1.1865, -0.5515,  1.6381,  ...,  1.0494,  0.7478,  0.8644],
         [-0.3774, -2.4135,  1.0490,  ...,  3.2198,  1.9540,  0.6144],
         [ 1.5739,  2.0039,  0.4442,  ...,  0.5678,  1.4503,  1.5932]],

        ...,

        [[-0.8121,  0.8864, -3.5894,  ..., -1.2631,  1.3639,  2.4222],
         [-0.0165, -1.9797, -0.97

<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
본 강의자료는 세바스찬 라시카(Sebastian Raschka)의 <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> (<a href="<a href="http://tensorflow.blog/llm-from-scratch">밑바닥부터 만들면서 배우는 LLM</a>)의 예제를 참고하여 제작되었습니다.
</font>
</td>
</tr>
</table>